In [ ]:
import numpy as np
import seaborn
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
import imageio
import glob
from IPython import display
from matplotlib import cm
import time

seaborn.set()
np.set_printoptions(suppress=True)
hsv_colors = cm.get_cmap('hsv')

import utils
import clustering_functions

In [ ]:
dataset1, dataset2, dataset3 = utils.read_dataset2()
dataset1, dataset2, dataset3 = utils.normalize_dataset2(dataset1, dataset2, dataset3)

color_list = ['firebrick','darkorange','darkgoldenrod','forestgreen','dodgerblue','blueviolet','magenta', 'black']
utils.plot_dataset2(dataset1, dataset2, dataset3, color_list)

In [ ]:
clustering_functions.kmeans(dataset1, k=7, dataname="dataset1", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset1.gif')

In [ ]:
clustering_functions.kmeans(dataset2, k=3, dataname="dataset2", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset2.gif')

In [ ]:
clustering_functions.kmeans(dataset3, k=2, dataname="dataset3", create_anim_file=True, color_list=color_list)
display.Image(filename='gif/kmeans_dataset3.gif')

In [ ]:
%timeit clustering_functions.kmeans(dataset1, k=7, dataname="dataset1", create_anim_file=False, print_output=False)

In [ ]:
%timeit clustering_functions.kmeans(dataset2, k=3, dataname="dataset2", create_anim_file=False, print_output=False)

In [ ]:
%timeit clustering_functions.kmeans(dataset3, k=2, dataname="dataset3", create_anim_file=False, print_output=False)

In [ ]:
def compute_distance_matrix(dataset):
    sm = np.array([np.square(dataset - dataset[i]).sum(axis=1) for i in range(len(dataset))])
    sm[np.identity(len(sm)) == 1] = np.inf
    return sm

In [ ]:
def merge_clusters(distance_matrix, clusters, len_dm):
    argmin_dm = np.argmin(distance_matrix)
    min_i, min_j = argmin_dm // len_dm, argmin_dm % len_dm
    cluster_min_i, cluster_min_j = clusters[min_i], clusters[min_j]
    if not cluster_min_i == cluster_min_j:
        cluster_eq_i = clusters == cluster_min_i
        cluster_eq_j = clusters == cluster_min_j
        temp_dm = distance_matrix[cluster_eq_i]
        temp_dm[:, cluster_eq_j] = np.inf
        distance_matrix[cluster_eq_i] = temp_dm    
        temp_dm = distance_matrix[cluster_eq_j]
        temp_dm[:, cluster_eq_i] = np.inf
        distance_matrix[cluster_eq_j] = temp_dm
        clusters[cluster_eq_j] = cluster_min_i
    else:
        print("Error")
    return distance_matrix, clusters

In [ ]:
def single_linkage(dataset, nb_of_clusters, dataname="data", create_anim_file=False, plot_every_iter=15, print_output=True):
    start_time = time.time()
    dm = compute_distance_matrix(dataset)
    clusters = np.arange(len(dataset))
    len_dm = len(dm)
    
    for i in range(len(dataset) - nb_of_clusters):
        dm, clusters = merge_clusters(dm, clusters, len_dm)
        
        unique_clusters = np.unique(clusters)
        if create_anim_file and (i % plot_every_iter == 0 or len(unique_clusters) < 8):
            if len(unique_clusters) < 8:
                color_arr = [color_list[np.where(unique_clusters == cluster)[0][0]] for cluster in clusters]
            else:
                color_arr = [hsv_colors(cluster/len(dataset)) for cluster in clusters]
            plt.scatter(dataset[:,0], dataset[:,1], c=color_arr)
            plt.title(f'{dataname}')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.savefig(f'gif/singlelinkage/{dataname}/{i:04d}.png')
            plt.close()
    
    squared_errors = 0
    for i in unique_clusters:
        cluster_centroid = dataset[clusters == i].mean(axis=0)
        squared_error = np.square(dataset[clusters == i] - cluster_centroid).sum()
        squared_errors += squared_error
        
    end_time = time.time()
    if print_output:
        print(f"Sum of squared errors for {dataname} (normalized) with single-linkage:{squared_errors:.4f}")
        print(f"Single-linkage for {dataname} took :{end_time - start_time:.3f} seconds")

In [ ]:
create_anim_file = True

In [ ]:
single_linkage(dataset1, 7, dataname="dataset1", create_anim_file=create_anim_file, plot_every_iter=50)

In [ ]:
single_linkage(dataset2, 3, dataname="dataset2", create_anim_file=create_anim_file, plot_every_iter=20)

In [ ]:
single_linkage(dataset3, 2, dataname="dataset3", create_anim_file=create_anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/singlelinkage_dataset1.gif'

    frames = []
    filenames = glob.glob('gif/singlelinkage/dataset1/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/singlelinkage_dataset1.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/singlelinkage_dataset2.gif'

    frames = []
    filenames = glob.glob('gif/singlelinkage/dataset2/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/singlelinkage_dataset2.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/singlelinkage_dataset3.gif'

    frames = []
    filenames = glob.glob('gif/singlelinkage/dataset3/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/singlelinkage_dataset3.gif'
display.Image(filename=anim_file)

In [ ]:
def compute_distance_matrix(dataset):
    sm = np.array([np.square(dataset - dataset[i]).sum(axis=1) for i in range(len(dataset))])
    sm[np.identity(len(sm)) == 1] = np.nan
    return sm

In [ ]:
def merge_clusters(distance_matrix, clusters, len_dm):
    nanargmin_dm = np.nanargmin(distance_matrix)
    min_i, min_j = nanargmin_dm // len_dm, nanargmin_dm % len_dm
    cluster_min_i, cluster_min_j = clusters[min_i], clusters[min_j]
    if not cluster_min_i == cluster_min_j:
        cluster_eq_i = clusters == cluster_min_i
        cluster_eq_j = clusters == cluster_min_j
        temp_dm = distance_matrix[cluster_eq_i]
        temp_dm[:, cluster_eq_j] = np.nan
        distance_matrix[cluster_eq_i] = temp_dm    
        temp_dm = distance_matrix[cluster_eq_j]
        temp_dm[:, cluster_eq_i] = np.nan
        distance_matrix[cluster_eq_j] = temp_dm
        clusters[cluster_eq_j] = cluster_min_i
        cluster_eq_i = clusters == cluster_min_i
        cluster_eq_j = clusters == cluster_min_j
        temp_dm = distance_matrix[cluster_eq_i]
        temp_dm[:] = np.max(temp_dm, axis=0)
        distance_matrix[cluster_eq_i] = temp_dm
        temp_dm = distance_matrix[:,cluster_eq_i]
        temp_dm[:] = np.expand_dims(np.max(temp_dm, axis=1), axis=1)
        distance_matrix[:,cluster_eq_i] = temp_dm
    else:
        print("Error")

    return distance_matrix, clusters

In [ ]:
def complete_linkage(dataset, nb_of_clusters, dataname="data", create_anim_file=False, plot_every_iter=15, print_output=True):
    start_time = time.time()
    dm = compute_distance_matrix(dataset)
    clusters = np.arange(len(dataset))
    len_dm = len(dm)
    
    for i in range(len(dataset) - nb_of_clusters):
        dm, clusters = merge_clusters(dm, clusters, len_dm)
        
        unique_clusters = np.unique(clusters)
        if create_anim_file and (i % plot_every_iter == 0 or len(unique_clusters) < 8):
            if len(unique_clusters) < 8:
                color_arr = [color_list[np.where(unique_clusters == cluster)[0][0]] for cluster in clusters]
            else:
                color_arr = [hsv_colors(cluster/len(dataset)) for cluster in clusters]
            plt.scatter(dataset[:,0], dataset[:,1], c=color_arr)
            plt.title(f'{dataname}')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.savefig(f'gif/completelinkage/{dataname}/{i:04d}.png')
            plt.close()
    
    squared_errors = 0
    for i in unique_clusters:
        cluster_centroid = dataset[clusters == i].mean(axis=0)
        squared_error = np.square(dataset[clusters == i] - cluster_centroid).sum()
        squared_errors += squared_error
        
    end_time = time.time()
    if print_output:
        print(f"Sum of squared errors for {dataname} (normalized) with complete-linkage:{squared_errors:.4f}")
        print(f"Complete-linkage for {dataname} took :{end_time - start_time:.3f} seconds")

In [ ]:
create_anim_file = True

In [ ]:
complete_linkage(dataset1, 7, dataname="dataset1", create_anim_file=create_anim_file, plot_every_iter=50)

In [ ]:
complete_linkage(dataset2, 3, dataname="dataset2", create_anim_file=create_anim_file, plot_every_iter=20)

In [ ]:
complete_linkage(dataset3, 2, dataname="dataset3", create_anim_file=create_anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/completelinkage_dataset1.gif'

    frames = []
    filenames = glob.glob('gif/completelinkage/dataset1/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/completelinkage_dataset1.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/completelinkage_dataset2.gif'

    frames = []
    filenames = glob.glob('gif/completelinkage/dataset2/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/completelinkage_dataset2.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/completelinkage_dataset3.gif'

    frames = []
    filenames = glob.glob('gif/completelinkage/dataset3/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/completelinkage_dataset3.gif'
display.Image(filename=anim_file)

In [ ]:
def compute_distance_matrix(dataset):
    sm = np.array([np.square(dataset - dataset[i]).sum(axis=1) for i in range(len(dataset))])
    sm[np.identity(len(sm)) == 1] = np.nan
    return sm

In [ ]:
def merge_clusters(distance_matrix, clusters, len_dm):
    nanargmin_dm = np.nanargmin(distance_matrix)
    min_i, min_j = nanargmin_dm // len_dm, nanargmin_dm % len_dm
    cluster_min_i, cluster_min_j = clusters[min_i], clusters[min_j]
    if not cluster_min_i == cluster_min_j:
        cluster_eq_i = clusters == cluster_min_i
        cluster_eq_j = clusters == cluster_min_j
        temp_dm = distance_matrix[cluster_eq_i]
        temp_dm[:, cluster_eq_j] = np.nan
        distance_matrix[cluster_eq_i] = temp_dm    
        temp_dm = distance_matrix[cluster_eq_j]
        temp_dm[:, cluster_eq_i] = np.nan
        distance_matrix[cluster_eq_j] = temp_dm
        clusters[cluster_eq_j] = cluster_min_i
        cluster_eq_i = clusters == cluster_min_i
        cluster_eq_j = clusters == cluster_min_j
        temp_dm = distance_matrix[cluster_eq_i]
        temp_dm[:] = np.mean(temp_dm, axis=0)
        distance_matrix[cluster_eq_i] = temp_dm
        temp_dm = distance_matrix[:,cluster_eq_i]
        temp_dm[:] = np.expand_dims(np.mean(temp_dm, axis=1), axis=1)
        distance_matrix[:,cluster_eq_i] = temp_dm
    else:
        print("Error")

    return distance_matrix, clusters

In [ ]:
def group_average(dataset, nb_of_clusters, dataname="data", create_anim_file=False, plot_every_iter=15, print_output=True):
    start_time = time.time()
    dm = compute_distance_matrix(dataset)
    clusters = np.arange(len(dataset))
    len_dm = len(dm)
    
    for i in range(len(dataset) - nb_of_clusters):
        dm, clusters = merge_clusters(dm, clusters, len_dm)
        
        unique_clusters = np.unique(clusters)
        if create_anim_file and (i % plot_every_iter == 0 or len(unique_clusters) < 8):
            if len(unique_clusters) < 8:
                color_arr = [color_list[np.where(unique_clusters == cluster)[0][0]] for cluster in clusters]
            else:
                color_arr = [hsv_colors(cluster/len(dataset)) for cluster in clusters]
            plt.scatter(dataset[:,0], dataset[:,1], c=color_arr)
            plt.title(f'{dataname}')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.savefig(f'gif/groupaverage/{dataname}/{i:04d}.png')
            plt.close()
    
    squared_errors = 0
    for i in unique_clusters:
        cluster_centroid = dataset[clusters == i].mean(axis=0)
        squared_error = np.square(dataset[clusters == i] - cluster_centroid).sum()
        squared_errors += squared_error
        
    end_time = time.time()
    if print_output:
        print(f"Sum of squared errors for {dataname} (normalized) with group average:{squared_errors:.4f}")
        print(f"Group average for {dataname} took :{end_time - start_time:.3f} seconds")

In [ ]:
create_anim_file = True

In [ ]:
group_average(dataset1, 7, dataname="dataset1", create_anim_file=create_anim_file, plot_every_iter=50)

In [ ]:
group_average(dataset2, 3, dataname="dataset2", create_anim_file=create_anim_file, plot_every_iter=20)

In [ ]:
group_average(dataset3, 2, dataname="dataset3", create_anim_file=create_anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/groupaverage_dataset1.gif'

    frames = []
    filenames = glob.glob('gif/groupaverage/dataset1/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/groupaverage_dataset1.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/groupaverage_dataset2.gif'

    frames = []
    filenames = glob.glob('gif/groupaverage/dataset2/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/groupaverage_dataset2.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/groupaverage_dataset3.gif'

    frames = []
    filenames = glob.glob('gif/groupaverage/dataset3/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(10):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=8)
    
anim_file = 'gif/groupaverage_dataset3.gif'
display.Image(filename=anim_file)

In [ ]:
def dbscan(dataset, min_pts, eps, create_anim_file=False, dataname="data"):
    start_time = time.time()
    eps_c = eps * eps

    core_point_list = []
    for point_index, cur_point in enumerate(dataset):
        points_within_eps = 0
        for other_point_index, other_point in enumerate(dataset):
            if point_index == other_point_index:
                continue
            if np.square(cur_point - other_point).sum() < eps_c:
                points_within_eps += 1
            if points_within_eps >= min_pts:
                core_point_list.append(point_index)
                break

    core_points = dataset[core_point_list]
    non_core_points = np.delete(dataset, core_point_list, axis=0)

    border_point_list = []
    for non_core_point_index, non_core_point in enumerate(non_core_points):
        for core_point in core_points:
            if np.square(non_core_point - core_point).sum() < eps_c:
                border_point_list.append(non_core_point_index)
                break

    border_points = non_core_points[border_point_list]
    noise_points = np.delete(non_core_points, border_point_list, axis=0)

    cur_cluster = -1
    clusters = np.full(len(dataset), cur_cluster)
    unique_clusters = np.unique(clusters)
    
    done = False
    i = 0
    while not done:
         
        if create_anim_file and len(unique_clusters) <= 8:
            color_arr = [color_list[cluster] for cluster in clusters]
            plt.scatter(dataset[:,0], dataset[:,1], c=color_arr)
            plt.title(f'{dataname}')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.savefig(f'gif/dbscan/{dataname}/{i:04d}.png')
            plt.close()
        
        
        occurence = np.nonzero(clusters[core_point_list] == -1)
        if len(occurence[0]) > 0:
            core_point_occurence = occurence[0][0]
            cur_cluster += 1
            clusters[core_point_list[core_point_occurence]] = cur_cluster

            core_point = dataset[core_point_list[core_point_occurence]]
            other_core_indices = []
            for other_point_index, other_point in enumerate(dataset):
                if clusters[other_point_index] == -1 and np.square(core_point - other_point).sum() < eps_c:
                    clusters[other_point_index] = clusters[core_point_list[core_point_occurence]]
                    if other_point_index in core_point_list:
                        other_core_indices.append(other_point_index)
            for other_core_index in other_core_indices:
                new_core_point = dataset[other_core_index]
                for other_point_index, other_point in enumerate(dataset):
                    if clusters[other_point_index] == -1 and np.square(new_core_point - other_point).sum() < eps_c:
                        clusters[other_point_index] = clusters[other_core_index]
                        if other_point_index in core_point_list:
                            other_core_indices.append(other_point_index)
        else:
            done = True
        
        i += 1
            
    unique_clusters = np.unique(clusters)
    squared_errors = 0
    for i in unique_clusters:
        if i == -1:
            continue
        cluster_centroid = dataset[clusters == i].mean(axis=0)
        squared_error = np.square(dataset[clusters == i] - cluster_centroid).sum()
        squared_errors += squared_error
    
    end_time = time.time()
    print(f"Sum of squared errors for {dataname} (normalized) with DBSCAN:{squared_errors:.4f}")
    print(f"DBSCAN for {dataname} took :{end_time - start_time:.3f} seconds")

In [ ]:
create_anim_file = True

In [ ]:
dbscan(dataset1, min_pts=230, eps=0.795, create_anim_file=create_anim_file, dataname="dataset1")

In [ ]:
dbscan(dataset2, min_pts=30, eps=0.4, create_anim_file=create_anim_file, dataname="dataset2")

In [ ]:
dbscan(dataset3, min_pts=45, eps=0.927, create_anim_file=create_anim_file, dataname="dataset3")

In [ ]:
if create_anim_file:
    anim_file = 'gif/dbscan_dataset1.gif'

    frames = []
    filenames = glob.glob('gif/dbscan/dataset1/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(3):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=2)
    
anim_file = 'gif/dbscan_dataset1.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/dbscan_dataset2.gif'

    frames = []
    filenames = glob.glob('gif/dbscan/dataset2/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(3):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=2)
    
anim_file = 'gif/dbscan_dataset2.gif'
display.Image(filename=anim_file)

In [ ]:
if create_anim_file:
    anim_file = 'gif/dbscan_dataset3.gif'

    frames = []
    filenames = glob.glob('gif/dbscan/dataset3/*.png')
    filenames = sorted(filenames)
    for i, filename in enumerate(filenames):
        frames.append(imageio.imread(filename))
    for i in range(3):
        frames.append(imageio.imread(filename))

    imageio.mimsave(anim_file, frames, 'GIF', fps=2)
    
anim_file = 'gif/dbscan_dataset3.gif'
display.Image(filename=anim_file)